<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP/DZ_3_NER_bert_ru_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Classification. Практическое задание (PJ)

Для закрепления материала модуля вам необходимо решить задачу NER для предоставленного датасета, используя любые доступные вам средства. Модель должна обучаться на файле `train.txt`, валидироваться на файле `dev.txt`, а её качество необходимо оценить на файле `test.txt`.

Для достижения наилучшего результата уделите внимание подбору гиперпарметров как в плане архитектуры, так и в плане обучения модели.

Критерии оценивания проекта:

- общее качество кода и следование PEP-8;
- использование рекуррентных сетей;
- использованы варианты архитектур, близкие к state of the art для данной задачи;
- произведен подбор гиперпараметров;
- использованы техники изменения learning rate (lr scheduler);
- использована адекватная задаче функция потерь;
- использованы техники регуляризации;
- корректно проведена валидация модели;
- использованы техники ensemble;
- использованы дополнительные данные;

- итоговое значение метрики качества > 0.6 (f1)

[датасет](https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip)

In [1]:
!pip install -U huggingface_hub -q

!pip install transformers -q
!pip install datasets -q

!pip install accelerate -U -q
!pip install transformers[torch] -q

!pip install seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


restart!!

In [1]:
import numpy as np
import pandas as pd

In [2]:
loaded_path = '/content/data.zip'
!wget https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip -O {loaded_path}


--2023-10-05 08:37:55--  https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip
Resolving lms.skillfactory.ru (lms.skillfactory.ru)... 51.250.7.2
Connecting to lms.skillfactory.ru (lms.skillfactory.ru)|51.250.7.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870425 (850K) [application/x-zip-compressed]
Saving to: ‘/content/data.zip’

/content/data.zip   100%[===================>] 850.02K   988KB/s    in 0.9s    

2023-10-05 08:37:57 (988 KB/s) - ‘/content/data.zip’ saved [870425/870425]



In [3]:
!unzip {loaded_path}

Archive:  /content/data.zip
  inflating: dev.txt                 
  inflating: test.txt                
  inflating: train.txt               


In [4]:
import json

def txt2json(file_path):

    save_path = f"{file_path.split('.')[0]}"+'.json'
    out_l = []
    with open(file_path, 'r') as in_file:

      stripped = (line.strip('\n') for line in in_file)
      line_dict = dict()

      txt = []
      lbl = []
      j = 0
      for i, line in enumerate(stripped):
        if line:
          t, l =  line.split(' ')
          txt.append(t)
          lbl.append(l)
        if not line:
          line_dict['tokens'] = txt
          line_dict['ner_tags'] = lbl
          out_l.append(line_dict)
          txt = []
          lbl = []
          line_dict = dict()
          j+=1

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(out_l, f, ensure_ascii=False, indent=4)





In [5]:
txt2json('train.txt')
txt2json('dev.txt')
txt2json('test.txt')

In [7]:
from datasets import load_dataset

In [108]:
# https://huggingface.co/docs/datasets/process#flatten

datasets = load_dataset("json",
                       data_files={"train": "train.json", 'valid': "dev.json", "test": "test.json"},
                       )

In [9]:
datasets['train'], datasets['valid'], datasets['test'], datasets['train'][2]

(Dataset({
     features: ['ner_tags', 'tokens'],
     num_rows: 7746
 }),
 Dataset({
     features: ['ner_tags', 'tokens'],
     num_rows: 2582
 }),
 Dataset({
     features: ['ner_tags', 'tokens'],
     num_rows: 2582
 }),
 {'ner_tags': ['O',
   'B-LOC',
   'I-LOC',
   'I-LOC',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'B-PER',
   'I-PER',
   'O'],
  'tokens': ['В',
   'Ханты-Мансийском',
   'автономном',
   'округе',
   'с',
   'должности',
   'снят',
   'начальник',
   'УВД',
   'Николай',
   'Гудожников',
   '.']})

In [10]:
from datasets import ClassLabel, Sequence
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,ner_tags,tokens
0,"[O, B-LOC, O, O, O, O, O, B-ORG, O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O, O, O, O]","[Президент, США, назначил, четырех, послов, в, обход, Сената, Президент, США, Барак, Обама, воспользовался, парламентскими, каникулами, ,, чтобы, назначить, четырех, послов, ,, сообщает, Associated, Press, в, среду, ,, 29, декабря, .]"
1,"[O, O, B-ORG, O, O, B-ORG, B-LOC, O, O, O, O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, O, O]","[Как, сообщили, РБК, в, пресс-центре, МВД, России, ,, уволенные, сегодня, президентом, России, Дмитрием, Медведевым, милицейские, генералы, покинули, должности, по, собственному, желанию, .]"
2,"[O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, B-PER, I-PER, O]","[Новым, директором, стал, ректор, Российского, государственного, университета, нефти, и, газа, имени, Губкина, Виктор, Мартынов, .]"
3,"[O, O, O, O, O, O, O, O, B-LOC, O]","[Об, этом, сообщили, в, пресс-службе, американского, посольства, в, РФ, .]"
4,"[O, O, B-LOC, I-LOC, O, O, O, O, O]","[в, акватории, Тихого, океана, ,, сообщает, пресс-служба, ведомства, .]"
5,"[O, O, O, O, O, O, O, O, O, O]","[Однако, кадровый, резерв, может, формироваться, именно, из, таких, людей, .]"
6,"[B-PER, O, O, O, O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Путин, во, вторник, проведет, совещание, по, вопросам, развития, ОАК, В, ходе, совещания, будут, обсуждаться, меры, по, развитию, гражданского, сегмента, российского, авиастроения, .]"
7,"[O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Ранее, ожидалось, ,, что, эмир, Кувейта, не, только, примет, отставку, правительства, ,, но, и, распустит, парламент, ,, однако, информация, о, новых, выборах, пока, не, получила, подтверждения, .]"
8,"[O, O, B-LOC, I-LOC, I-LOC, O, O, O, O, B-ORG, O, B-PER, I-PER, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, B-PER, I-PER, O]","[Среди, руководителей, Московского, военного, округа, в, отставку, ушли, начальник, ПВО, полковник, Глеб, Еремин, ,, начальник, штаба, тыла, полковник, Олег, Князев, ,, глава, службы, ракетно-артиллерийского, вооружения, округа, Борис, Овсянников, и, начальник, штаба, вооружения, Александр, Анистратенко, .]"
9,"[B-ORG, I-ORG, I-ORG, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, B-ORG, O, O]","[Общероссийский, народный, фронт, обратился, к, депутатам, Госдумы, с, просьбой, не, поддерживать, позицию, комитета, по, труду, ,, соцполитике, и, делам, ветеранов, относительно, законопроекта, о, страховых, взносах, для, предпринимателей, и, поставить, вопрос, о, соответствии, занимаемой, должности, председателя, комитета, единоросса, Андрея, Исаева, ,, говорится, в, размещенном, на, сайте, ОНФ, сообщении, .]"


# Dataset Object without ClassLabel to ClassLabel
https://discuss.huggingface.co/t/dataset-object-without-classlabel/33029

In [65]:
def flat_lists(seqs):
  out = []
  for s in seqs:
    out.extend(s)
  return out

In [66]:
label_names = list(set(flat_lists(datasets["train"]['ner_tags'])))
NUM_CLASSES = len(label_names)
print(f'NUM_CLASSES = {NUM_CLASSES}')

dict_lbl_to_ner = {k: v for k, v in enumerate(label_names)}
dict_ner_to_lbl = {v: k for k, v in dict_lbl_to_ner.items()}
print(f'NER to lbl dict - {dict_ner_to_lbl}')
label_names

NUM_CLASSES = 7
NER to lbl dict - {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}


[0, 1, 2, 3, 4, 5, 6]

In [109]:
datasets["train"].features['ner_tags']

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

Correct problem

In [110]:
datasets = datasets.cast_column("ner_tags", Sequence(ClassLabel(names=label_names)))

In [111]:
datasets["train"].features

{'ner_tags': Sequence(feature=ClassLabel(names=['I-PER', 'I-ORG', 'B-PER', 'B-LOC', 'I-LOC', 'O', 'B-ORG'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [68]:
ner_feature = datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['I-PER', 'I-ORG', 'B-PER', 'B-LOC', 'I-LOC', 'O', 'B-ORG'], id=None), length=-1, id=None)

In [69]:
label_names = ner_feature.feature.names
label_names

['I-PER', 'I-ORG', 'B-PER', 'B-LOC', 'I-LOC', 'O', 'B-ORG']

In [70]:
words = datasets["train"][0]["tokens"]
labels = datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

" Если Миронов занял столь оппозиционную позицию , то мне представляется , что для него было бы порядочным и правильным уйти в отставку с занимаемого им поста , поста , который предоставлен ему сегодня " Единой Россией '' и никем больше '' , - заключает Исаев . 
O O    B-PER   O     O     O             O       O O  O   O              O O   O   O    O    O  O          O O          O    O O        O O           O  O     O O     O O       O            O   O       O B-ORG  I-ORG   O  O O     O      O  O O O         B-PER O 


#tokenizer

In [112]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
model_checkpoint = "KoichiYasuoka/bert-base-russian-upos"
task = "ner" # Should be one of "ner", "pos" or "chunk"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [72]:
example = datasets["train"][4]
print(example["tokens"])

['Для', 'этого', 'ей', 'пришлось', 'выиграть', 'выборы', 'в', 'муниципальном', 'округе', '"', 'Красненькая', 'речка', "''", '.']


In [113]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Для', 'этого', 'ей', 'пришлось', 'выиграть', 'выборы', 'в', 'муниципальном', 'округе', '"', 'Красн', '##енькая', 'речка', "'", "'", '.', '[SEP]']


In [74]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(14, 27)

In [75]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, 5, 6, 7, 7, 8, 9, 10, 10, 10, 10, 10, 11, 11, 12, 12, 13, None]


In [114]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


In [18]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [77]:
tokenizer.is_fast

True

In [115]:
inputs = tokenizer(datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '"',
 'Если',
 'Миронов',
 'занял',
 'столь',
 'оппозиционную',
 'позицию',
 ',',
 'то',
 'мне',
 'представляется',
 ',',
 'что',
 'для',
 'него',
 'было',
 'бы',
 'поряд',
 '##очным',
 'и',
 'правильным',
 'уйти',
 'в',
 'отставку',
 'с',
 'заним',
 '##аемого',
 'им',
 'поста',
 ',',
 'поста',
 ',',
 'который',
 'предоставлен',
 'ему',
 'сегодня',
 '"',
 'Единой',
 'Россией',
 "'",
 "'",
 'и',
 'никем',
 'больше',
 "'",
 "'",
 ',',
 '-',
 'заключает',
 'Исаев',
 '.',
 '[SEP]']

In [79]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 2,
 3,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 17,
 17,
 17,
 18,
 19,
 19,
 20,
 20,
 21,
 22,
 23,
 24,
 24,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 31,
 31,
 31,
 32,
 33,
 34,
 35,
 35,
 35,
 36,
 36,
 36,
 37,
 37,
 38,
 39,
 39,
 39,
 40,
 41,
 41,
 42,
 43,
 44,
 44,
 44,
 44,
 45,
 45,
 45,
 46,
 None]

In [80]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [116]:
labels = datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5]
[-100, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 1, 5, 6, 5, 5, 5, 5, 6, 5, 5, 5, 2, 5, -100]


In [82]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [117]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 108, 10830, 37027, 11532, 18488, 119392, 18099, 128, 3815, 16740, 34246, 128, 1997, 2748, 7268, 3332, 1655, 10040, 20422, 851, 64804, 22096, 845, 14800, 869, 5358, 17097, 2718, 18876, 128, 18876, 128, 3856, 64378, 6164, 12904, 108, 25285, 17814, 118, 118, 851, 94639, 8980, 118, 118, 128, 130, 54498, 63375, 132, 102], [101, 29812, 108, 829, 118, 118, 845, 35958, 43407, 36122, 13012, 116338, 128, 1997, 57441, 118364, 878, 39781, 869, 11211, 108, 25699, 15703, 869, 38637, 866, 118, 118, 132, 102], [101, 781, 32681, 130, 86858, 50883, 14656, 869, 11995, 13108, 14364, 34775, 11843, 20721, 1771, 3636, 132, 102], [101, 811, 10641, 1768, 1516, 8092, 3299, 4415, 13796, 23613, 22117, 9095, 108, 115385, 11299, 7604, 118, 118, 186, 130, 69820, 183, 132, 102], [101, 7638, 4638, 10999, 14384, 25299, 15749, 845, 40535, 14656, 108, 111315, 98341, 60234, 118, 118, 132, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [118]:
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

Map:   0%|          | 0/7746 [00:00<?, ? examples/s]

Map:   0%|          | 0/2582 [00:00<?, ? examples/s]

Map:   0%|          | 0/2582 [00:00<?, ? examples/s]

# DataCollatorForTokenClassification

In [119]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [120]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    5,    5,    2,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    6,    1,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    2,    5, -100],
        [-100,    5,    5,    6,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    2,    2,    5,    5,    5,    5,    5,    5,    5,
            3,    3,    5,    5,    5, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100]])

In [121]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, -100]
[-100, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, -100]


# Evaluate

In [95]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.4 MB/s eta 0:00:00


In [96]:
import evaluate
metric = evaluate.load("seqeval")

In [122]:
labels = datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'O']

In [123]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.9787234042553191}

In [144]:
labels

tensor([[-100,    5,    5,    5,    5,    3,    4,    5,    2,    0,    5,    5,
            5,    5,    5,    5,    5,    5,    5, -100, -100, -100, -100, -100,
         -100, -100],
        [-100,    5,    5,    3,    5, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100],
        [-100,    5,    5,    5,    5,    5,    6,    1,    1,    1,    1,    5,
            1,    1,    1,    1,    1,    1,    1,    1,    5,    5, -100, -100,
         -100, -100],
        [-100,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    2,
            5,    5,    5,    5, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100],
        [-100,    5,    5,    5,    5,    5,    6,    1,    5,    5,    6,    1,
            1,    1,    1,    6,    1,    6,    3,    5,    5,    2,    0,    0,
            5, -100],
        [-100,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,

In [143]:
predictions

tensor([[5, 5, 5, 5, 5, 3, 4, 5, 2, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5],
        [5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5],
        [5, 5, 5, 5, 5, 5, 6, 1, 1, 1, 1, 5, 1, 3, 3, 1, 3, 3, 1, 1, 5, 5, 5, 5,
         5, 5],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5],
        [5, 5, 5, 5, 5, 5, 6, 1, 5, 5, 6, 1, 1, 1, 1, 6, 1, 6, 3, 5, 5, 2, 0, 0,
         5, 5],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 2, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5]], device='cuda:0')

In [100]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [124]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [184]:
label2id, id2label

({'I-PER': 0,
  'I-ORG': 1,
  'B-PER': 2,
  'B-LOC': 3,
  'I-LOC': 4,
  'O': 5,
  'B-ORG': 6},
 {0: 'I-PER',
  1: 'I-ORG',
  2: 'B-PER',
  3: 'B-LOC',
  4: 'I-LOC',
  5: 'O',
  6: 'B-ORG'})

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [131]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], collate_fn=data_collator, batch_size=8
)

In [133]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at KoichiYasuoka/bert-base-russian-upos and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([89, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([89]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [134]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [135]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [136]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [137]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [141]:
output_dir = "content/"
#repo = Repository(output_dir, clone_from=repo_name)

In [142]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))
output_dir = '/content'

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
          #  commit_message=f"Training in progress epoch {epoch}", blocking=False
       # )

  0%|          | 0/2907 [00:00<?, ?it/s]

epoch 0: {'precision': 0.9597724502234863, 'recall': 0.94948412166689, 'f1': 0.954600565808972, 'accuracy': 0.9910234286852061}
epoch 1: {'precision': 0.9804957334416904, 'recall': 0.979301948051948, 'f1': 0.9798984771573604, 'accuracy': 0.994823123382226}
epoch 2: {'precision': 0.9813084112149533, 'recall': 0.9777327935222672, 'f1': 0.9795173392820927, 'accuracy': 0.9952047521072542}


# Check test data

In [148]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

test_dataloader = accelerator.prepare(test_dataloader)

In [169]:
# Evaluation
all_predictions = []
all_labels = []
model.eval()
for batch in test_dataloader:
    with torch.no_grad():
        outputs = model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)

    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=true_predictions, references=true_labels)

    all_predictions.extend(list(predictions.cpu().numpy().flatten()))
    all_labels.extend(list(labels.cpu().numpy().flatten()))

results = metric.compute()
print(
    f"epoch {epoch}:",
    {
        key: results[f"overall_{key}"]
        for key in ["precision", "recall", "f1", "accuracy"]
    },
)

epoch 2: {'precision': 0.9801450935471554, 'recall': 0.978401727861771, 'f1': 0.9792726347914548, 'accuracy': 0.9952444999519646}


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

precision, recall, f1score = precision_recall_fscore_support(labels, predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.8030295362386545, recall: 0.746459171229349, f1score: 0.7581487587875906


In [164]:
from sklearn.metrics import classification_report

In [191]:
all_labels = np.array(all_labels)
all_labels[all_labels==-100] = 0

In [192]:
print(classification_report(all_labels, all_predictions, target_names=label_names))

              precision    recall  f1-score   support

       I-PER       1.00      0.05      0.09     68737
       I-ORG       0.56      0.96      0.71      1236
       B-PER       0.66      1.00      0.79      3606
       B-LOC       0.79      0.98      0.87      1852
       I-LOC       0.60      0.98      0.74       355
           O       0.45      1.00      0.62     50839
       B-ORG       0.70      0.97      0.82      2305

    accuracy                           0.49    128930
   macro avg       0.68      0.85      0.66    128930
weighted avg       0.76      0.49      0.35    128930

